In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *

import tensorflow as tf

Using TensorFlow backend.


In [2]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/chars.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(x_train.max())
print('max id {}'.format(max_idx))

max len seq 238
max id 126


In [7]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [8]:
x_train_pad.shape

(7410, 238)

In [9]:
x_train_pad[:2]

array([[28,  6,  7,  3,  6,  7,  4,  1, 10,  3,  1, 17,  3,  8,  7,  2,  7,
         3,  1, 17,  5,  8,  1, 50,  2,  3,  1, 21,  3, 36,  2,  4,  6,  3,
        10,  5,  1, 11,  2,  1, 12,  4, 18, 17, 10,  2, 12,  3, 22,  1,  9,
        15, 13, 24, 15, 19, 13, 26,  9,  1, 14,  2, 12,  5,  1, 12, 29,  5,
         1, 47, 18,  2, 12,  3,  1,  7, 16,  7,  7,  2,  1, 11,  4,  6,  4,
         1, 20,  3,  8,  2,  4,  1,  9, 15, 13, 24, 15, 19, 13, 26,  9, 47,
         1,  9, 34, 31, 33,  9,  1, 23,  2,  3,  1,  9, 20, 27, 21, 19, 28,
        30, 21,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [11]:
callbacks = [
    TensorBoard(log_dir='./logs_char_lstm', histogram_freq=1, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
    ReduceLROnPlateau(patience=2, verbose=1)
]

Instructions for updating:
Use the retry module or similar alternatives.


In [12]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx, output_dim=64, input_length=max_len_seq))
    model.add(LSTM(64,  dropout=0.0, recurrent_dropout=0.0, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [13]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 238, 64)           8064      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 41,153
Trainable params: 41,153
Non-trainable params: 0
_________________________________________________________________


In [18]:
!rm -rf logs_word_lstm
!mkdir -p logs_word_lstm

In [14]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=32, epochs=10, callbacks=callbacks)

Train on 5928 samples, validate on 1482 samples
Epoch 1/10
5928/5928 [==============================] - 47s 8ms/step - loss: 0.6258 - acc: 0.6883 - val_loss: 0.6205 - val_acc: 0.6883
Epoch 2/10
5928/5928 [==============================] - 47s 8ms/step - loss: 0.6207 - acc: 0.6883 - val_loss: 0.6207 - val_acc: 0.6883
Epoch 3/10
5928/5928 [==============================] - 47s 8ms/step - loss: 0.6213 - acc: 0.6884 - val_loss: 0.6206 - val_acc: 0.6883
Epoch 4/10
5928/5928 [==============================] - 47s 8ms/step - loss: 10.2391 - acc: 0.0523 - val_loss: 10.9725 - val_acc: 0.0000e+00


InvalidArgumentError: Nan in summary histogram for: lstm_1/kernel_0
	 [[Node: lstm_1/kernel_0 = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](lstm_1/kernel_0/tag, lstm_1/kernel/read/_131)]]

Caused by op 'lstm_1/kernel_0', defined at:
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/fabio/miniconda3/envs/dl/lib/python3.6/site-package

In [ ]:
del model

## Evaluate on the test set

In [ ]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
]

In [ ]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=6, callbacks=callbacks)

In [ ]:
loss, acc = model.evaluate(x_test_pad, y_test)

In [ ]:
print('loss {}, acc {}'.format(loss, acc))

In [ ]:
model.save('models/sentipolc_char_lstm.hdf5')